# Run a Test simulation

In [1]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st
import itertools as it

from tqdm.auto import tqdm

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [2]:
directory = "/media/Antonio/Dynamic/DimerCurrents/TestDataset"

In [3]:
os.system("rm -r %s"%directory)

0

In [4]:
h = 3.9*ureg.um
rho = 1.4 * ureg.um

region, positions = mgc.ordered_dimers(
    n_of_particles = 250,
    packing=0.35, height = h.magnitude,
    r = rho.magnitude, theta = 0)

particles = mgc.particles(
    positions*ureg.um,
    radius = 1.4*ureg.um,
    susceptibility = 0.4,
    diffusion=0.073*ureg.um**2/ureg.s,
    density = 1000*ureg.kg/ureg.m**3,
    temperature=300*ureg.K)

In [5]:
def redefine_field(field,tilt_angle):
    field.fieldx = 'v_Bmag*(sin(v_freq*time*2*PI)*sin(v_theta)+cos(v_theta)*%s)'%np.sin(tilt_angle).magnitude
    field.fieldy = 'v_Bmag*(cos(v_freq*time*2*PI)*sin(v_theta))'%np.sin(tilt_angle).magnitude
    field.fieldz = 'v_Bmag*(cos(v_theta)*%s)'%np.cos(tilt_angle).magnitude
    return field

In [6]:
world = mgc.world(particles, temperature = 300*ureg.K,
                  lj_parameters = [1e-2*ureg.pg*ureg.um**2/ureg.us**2, 2**(-1/6)],
                  region=region*ureg.um, boundaries = ['p','p','f'], 
                  walls = [False,False,True],
                  dipole_cutoff = 30*ureg.um)

total_time = 1*ureg.s
tilt_angle = np.arange(0,10,1)

In [7]:
index_name = "index.dat"

import csv

def prepare_simulation(delta, seed):
    
    field = mgc.field(magnitude = 7.28*ureg.mT, frequency = 3*ureg.Hz,
                  angle = 27*ureg.degrees, multibody_iter=2)
    
    field = redefine_field(field,delta*ureg.deg)

    sim = mgc.sim(dir_name = directory,
                  file_name="Tilt_%s_sd_%s_deg_h%u"%(delta,seed,h.to(ureg.nm).magnitude),
                  stamp_time=True,
                  timestep = 1e-4*ureg.s, framerate = 60*ureg.Hz, total_time = total_time,
                  particles = particles, world = world, 
                  field = field, output=["x","y","z","mux","muy","muz","fx","fy"],
                  seed = seed)
    
    sim.tilt = delta
    sim.height = h
    
    return sim

In [8]:
simulations = [prepare_simulation(delta, seed)
               for delta, seed in it.product(tilt_angle, 12345 + np.arange(3))]

In [9]:
def run_simulation(sim):
    
    sim = sim
    seed = sim.seed
    
    import csv
    import os
    index_name = "index.dat"
    directory = sim.dir_name
        
    sim.generate_scripts()
    out = sim.run(verbose=False)

    sim.load()
    max_frame = max(sim.lazy_read.T.keys())
    
    h = sim.height
    delta = sim.tilt
        
    with open(os.path.join(directory,index_name),'a',newline='') as file:
        writer = csv.writer(file,delimiter=' ')
        writer.writerow([os.path.split(sim.base_name)[-1],
                         h.magnitude, delta, seed, max_frame])


    return out

In [10]:
#out = run_simulation(simulations[0])

## Run Asynchronously

In [11]:
from multiprocessing import Pool

In [12]:
%%time
if __name__ ==  '__main__': 
    num_processors = 30
    p = Pool(processes = num_processors)
    as_res = p.map(run_simulation, simulations)

CPU times: user 34.2 ms, sys: 89.2 ms, total: 123 ms
Wall time: 27.5 s


In [13]:
index = pd.read_csv(os.path.join(directory,"index.dat"),
                    names = ["file", "height", "tilt", "exp", "max_frame"],
                    sep=" ")
index

,file,height,tilt,exp,max_frame
0,Tilt_1_sd_12347_deg_h3899_2023_08_15_15_08_12,3.9,1,12347,9853
1,Tilt_9_sd_12345_deg_h3899_2023_08_15_15_08_12,3.9,9,12345,9853
2,Tilt_8_sd_12345_deg_h3899_2023_08_15_15_08_12,3.9,8,12345,9853
3,Tilt_6_sd_12345_deg_h3899_2023_08_15_15_08_12,3.9,6,12345,9853
4,Tilt_3_sd_12346_deg_h3899_2023_08_15_15_08_12,3.9,3,12346,9853
5,Tilt_4_sd_12346_deg_h3899_2023_08_15_15_08_12,3.9,4,12346,9853
6,Tilt_2_sd_12345_deg_h3899_2023_08_15_15_08_12,3.9,2,12345,9853
7,Tilt_8_sd_12346_deg_h3899_2023_08_15_15_08_12,3.9,8,12346,9853
8,Tilt_7_sd_12347_deg_h3899_2023_08_15_15_08_12,3.9,7,12347,9853
9,Tilt_6_sd_12346_deg_h3899_2023_08_15_15_08_12,3.9,6,12346,9853
